Note: Briefly switched to pdfplumber to parse sections using text metadata (fontname: bold) instead because section 3.1 title does not agree with title formatting conventions in the rest of the document. The title is not capitalized but it is bold indicating that it is a title. 

In [163]:
import pandas as pd
import csv
import re
from PyPDF2 import PdfReader
import pdfplumber

Read PDF, correct format errors in extracted page text, convert to CSV format data (cell per word), and initialize dataframe with input data with the associated page number and row index by line.

In [164]:
def correct_format_issues(page_text, page_number):
    # Combine all patterns into a single regex using named groups
    all_patterns_regex = (
        r'(?P<line_break>[a-z][A-Z])|'  
        r'(?P<header>R1-1\S+)|'         
        r'(?P<footer_1>City of Vancouver.*)|'  
        rf'(?P<footer_2>.*Page\s+{page_number})'
    )
    
    # Correct all formatting issues found in a single pass
    for match in re.finditer(all_patterns_regex, page_text):
        if match.group('line_break'):
            # Add line breaks for line break issues
            page_text = re.sub(r'([a-z])([A-Z])', r'\1\n\2', page_text)
        else:
            # Delete headers and footers from the document
            page_text = re.sub(r'R1-1', '', page_text)
            page_text = re.sub(re.escape(match.group(0)), '', page_text)

    return page_text


In [165]:
pdf_file_name = 'zoning-by-law-district-schedule-r1-1.pdf'
# Read PDF & correct formatting issues
reader = PdfReader(pdf_file_name)
pdf_text = []

for page_number, page in enumerate(reader.pages, start=1):
    # Correct formatting issues before appending page text   
    page_text = correct_format_issues(page.extract_text(), page_number)
    #  print(f'Page Text Being Appended: {page_text}')
    pdf_text.append(page_text)

# Store document text in cell format, skip empty or blank lines
# Record page number associated with each line
csv_format_data = []
line_page_numbers = []
for page_number, page_text in enumerate(pdf_text, start=1):
    lines = page_text.splitlines()
    for line_index, line in enumerate(lines):
        if line:
            line_split = line.split()
            csv_format_data.append(line_split)
            line_page_numbers.append(page_number)

df_org = pd.DataFrame(csv_format_data)
df_org.insert(0, 'Page Number', line_page_numbers)
df_org.insert(1, 'Row Index', range(1, len(df_org) + 1))
# Parquet keeps None values instead of replacing with NaN on read
df_org.to_parquet('csv_format_data.parquet', index=False)

In [166]:
# Initialize DF with CSV formatted data for processing
df_org = pd.read_parquet('csv_format_data.parquet')
data_column_1 = df_org.iloc[:, 2]

# Initialize dictionary / vectors to hold DF column vectors to avoid repeat DF access.
all_sections = []
formatted_columns = {
    'Parent': [],
    'Section': [],
    'Section Title': [],
    'Section Start Page': [],
    'Row Index': []
}

# Text that may appear lowercase in section titles 
conjunctions = [
    'and', 'the', 'of', 'to', 'in', 'a', 'for', 'on', 'with', 'as', 'at', 
    'by', 'but', 'or', 'an', 'so', 'if', 'because', 'that', 'while', 'from', 
    'after', 'when', 'before', 'where', 'nor', 'yet', 'once', 'although',
    'since', 'than', 'though', 'unless', 'until', 'up', 'while'
]

def get_section_title(section_level, row):
    section_title = ''

    for cell in row:
        # End of section title text
        if cell is None:
            break
        # Not a section title, returns empty string
        if section_level == 0 and not cell.isupper(): 
            return ''
        if section_level > 1 and not cell[0].isupper() and cell not in conjunctions: 
            return ''
        section_title += f'{cell} '

    return section_title.strip()
        
def add_section_row(parent_section, section_number, section_title, section_page_start, section_row_index):
    formatted_columns['Parent'].append(parent_section)
    formatted_columns['Section'].append(section_number)
    formatted_columns['Section Title'].append(section_title)
    formatted_columns['Section Start Page'].append(section_page_start)
    formatted_columns['Row Index'].append(section_row_index)
    
def get_section_range(sections, idx):
    section_start = sections[idx]
    section_end = sections[idx + 1] if idx + 1 < len(sections) else len(df_org)
    return section_start, section_end

def get_section_data(section_level, parent_section, section_start, section_end):
    subsection = 1
    sections = []

    for row_idx in range(section_start, section_end):
        if section_level == 0: 
            next_section = f'{subsection}'
        else: 
            next_section = f'{parent_section}.{subsection}'
        
        section_number = data_column_1[row_idx]
        if section_number.strip() == next_section:
            row = df_org.iloc[row_idx]
            section_title = get_section_title(section_level, row[3:])
            if section_title: 
                section_page_start = row['Page Number']
                add_section_row(parent_section, section_number, section_title, section_page_start, row_idx)
                sections.append(row_idx)
                subsection += 1
                
    return sections

def get_sections_body(section_row_idxs):
    all_sections_body = []

    # Iterate through sections using enumerate on row_index
    for section_idx, current_row_index in enumerate(section_row_idxs):
        start_row = current_row_index + 1
        end_row = section_row_idxs[section_idx + 1] if section_idx + 1 < len(section_row_idxs) else len(df_org)

        # Extract rows between current and next row index
        section_body_rows = df_org.iloc[start_row:end_row, 2:]
        section_body_lines = []

        for _, row in section_body_rows.iterrows():
            row_text = ''
            for cell in row:
                if cell is None:
                    break
                row_text += f'{cell} '
            section_body_lines.append(row_text.strip())

        # Combine all rows into a single string with newlines
        section_body_text = '\n'.join(section_body_lines)
        all_sections_body.append(section_body_text)
        
    return all_sections_body

def get_sections_end_page(section_start_pages):
    section_end_pages = []

    for idx in range(len(section_start_pages)):
        if idx + 1 < len(section_start_pages):
            section_end_pages.append(section_start_pages[idx + 1])
        else:
            section_end_pages.append(section_start_pages[idx])

    return section_end_pages

def generate_csv_format():
    # Create DataFrame from formatted_columns
    df_final = pd.DataFrame(formatted_columns)
    df_final = df_final.sort_values(by='Row Index', ascending=True)

    # Extract body text by section row indices
    section_row_idxs = df_final['Row Index'].tolist()
    sections_body = get_sections_body(section_row_idxs)

    # Insert 'Section Body Text' column after 'Section Title'
    body_column_idx = df_final.columns.get_loc('Section Title') + 1
    df_final.insert(body_column_idx, 'Section Body Text', sections_body)

    # Extract end pages from following section start page
    section_start_pages = df_final['Section Start Page'].tolist()
    section_end_pages = get_sections_end_page(section_start_pages)

    # Insert 'Section End Page' column after 'Section Start Page'
    end_page_idx = df_final.columns.get_loc('Section Start Page') + 1
    df_final.insert(end_page_idx, 'Section End Page', section_end_pages)

    df_final = df_final.drop(columns=['Row Index'])
    return df_final

Begin parsing document by extracting only main section heading values and their indices.  
Iterate through main sections to find all subsections.

In [167]:
all_sections = []
# Initial pass through the original data to extract main section row indices
main_sections = get_section_data(0, None, 0, len(df_org))
all_sections.append(main_sections)
all_sections_idx = 0

for section_level, current_section in enumerate(all_sections, start=1):
    next_level_sections = []
    for section_idx, row_idx in enumerate(current_section):
        parent_section = formatted_columns['Section'][all_sections_idx]
        section_start, section_end = get_section_range(current_section, section_idx)
        subsections = get_section_data(section_level, parent_section, section_start, section_end)
        next_level_sections.extend(subsections)
        all_sections_idx += 1

    if next_level_sections: 
        all_sections.append(next_level_sections)
        
# Generate the final CSV format
converted_data = generate_csv_format()
converted_data.to_csv('final-csv-conversion.csv', index=False)
converted_data

,Parent,Section,Section Title,Section Body Text,Section Start Page,Section End Page
0,None,1,INTENT AND OVERVIEW,,1,1
4,1,1.1,Intent,The intent of this Residential Inclusive distr...,1,1
5,1,1.2,Overview,The table below provides an overview of the ou...,1,2
1,None,2,USE REGULATIONS,,2,2
6,2,2.1,Outright and Conditional Approval Uses,All outright and conditional approval uses are...,2,4
7,2,2.2,Use-Specific Regulations,"2.2.1 A new multiple dwelling , duplex with se...",4,7
2,None,3,"DENSITY, FORM AND PLACEMENT REGULATIONS","This section contains density, form and placem...",7,7
8,3,3.1,"Multiple dwelling , containing no more than 8 ...",Multiple dwelling containing no more than 8 dw...,7,7
14,3.1,3.1.1,Density and Floor Area,"3.1.1.1 The maximum floor space ratio is 0.70,...",7,8
15,3.1,3.1.2,Building Form and Placement,Regulations\n3.1.2.1 Site area and site fronta...,8,11
